In [1]:
input_file = "input_files/day_21.txt"

with open(input_file) as lines:
    data = lines.read().splitlines()

In [2]:
from collections import deque, namedtuple
from typing import NamedTuple


class Point(NamedTuple):
    row: int
    col: int

    def __add__(self, other):
        return Point(self.row + other.row, self.col + other.col)

def find_start(data):
    for row, line in enumerate(data):
        for col, char in enumerate(line):
            if char == "S":
                return Point(row, col)


In [3]:
data_ex = [
    '...........',
    '.....###.#.',
    '.###.##..#.',
    '..#.#...#..',
    '....#.#....',
    '.##..S####.',
    '.##..#...#.',
    '.......##..',
    '.##.#.####.',
    '.##..##.##.',
    '...........',
]

data_ex_large= [
    '.................................',
    '.....###.#......###.#......###.#.',
    '.###.##..#..###.##..#..###.##..#.',
    '..#.#...#....#.#...#....#.#...#..',
    '....#.#........#.#........#.#....',
    '.##...####..##..N####..##...####.',
    '.##..#...#..##..#...#..##..#...#.',
    '.......##.........##.........##..',
    '.##.#.####..##.#.####..##.#.####.',
    '.##..##.##..##..##.##..##..##.##.',
    '.................................',
    '.................................',
    '.....###.#......###.#......###.#.',
    '.###.##..#..###.##..#..###.##..#.',
    '..#.#...#....#.#...#....#.#...#..',
    '....#.#........#.#........#.#....',
    '.##...####..##..S####..##...####.',
    '.##..#...#..##..#...#..##..#...#.',
    '.......##.........##.........##..',
    '.##.#.####..##.#.####..##.#.####.',
    '.##..##.##..##..##.##..##..##.##.',
    '.................................',
    '.................................',
    '.....###.#......###.#......###.#.',
    '.###.##..#..###.##..#..###.##..#.',
    '..#.#...#....#.#...#....#.#...#..',
    '....#.#........#.#........#.#....',
    '.##...####..##...####..##...####.',
    '.##..#...#..##..#...#..##..#...#.',
    '.......##.........##.........##..',
    '.##.#.####..##.#.####..##.#.####.',
    '.##..##.##..##..##.##..##..##.##.',
    '.................................',
]

# Part One

In [4]:
def draw_ascii(data, point_set):    
    for row, line in enumerate(data):
        for col, char in enumerate(line):
            if Point(row, col) in point_set:
                print("O", end='')
            else:
                print(char, end='')
        print()
        
def get_valid(point, data):
    directions = [
        Point(1, 0),
        Point(-1, 0),
        Point(0, 1),
        Point(0, -1),
    ]
    for p in directions:
        pp = point + p
        if 0 <= pp.row < len(data) and 0 <= pp.col < len(data[0]) and data[pp.row][pp.col] != '#':
            yield pp

def bfs(data, start, steps):
    q = deque([(start, 0)])
    stopped = set()
    seen = set()
    while len(q):
        (current, depth) = q.popleft()
        if depth == 0 or (depth % 2 == 0):        
            stopped.add(current)

        for next_point in get_valid(current, data):
            if next_point not in seen:
                if depth < steps:
                    q.append((next_point, depth+1))
                    seen.add(next_point)

    return stopped
    


In [5]:
start = find_start(data_ex)

s = bfs(data_ex, start,6)
draw_ascii(data_ex, s)

...........
.....###.#.
.###.##.O#.
.O#O#O.O#..
O.O.#.#.O..
.##O.O####.
.##.O#O..#.
.O.O.O.##..
.##.#.####.
.##O.##.##.
...........


In [6]:
start = find_start(data)
s = bfs(data, start, 64)
print("Part One: ", len(s))

Part One:  3746


In [7]:
def get_valid_infinite(point, data):
    '''
    Don't check bounds just scale of map.
    '''
    directions = [
        Point(1, 0),
        Point(-1, 0),
        Point(0, 1),
        Point(0, -1),
    ]
    for p in directions:
        pp = point + p
        ck = Point(pp.row % len(data), pp.col % len(data[0]))
        if data[ck.row][ck.col] != '#':
            yield pp


def bfs(data, start, steps):
    '''
    Same as above, but don't check depth and look for
    odd steps instead of even
    '''
    q = deque([(start, 0)])
    stopped = set()
    seen = set()
    count = 0
    while len(q):
        count += 1
        (current, depth) = q.popleft()
        if depth == 0 or (depth % 2 == 1):
            stopped.add(current)

        for next_point in get_valid_infinite(current, data):
            if next_point not in seen:
                if depth < steps:
                    q.append((next_point, depth+1))
                seen.add(next_point)

    return stopped


### Requested number of steps `26501365`:

This reduces to $N * mapWidth + edgeDistance$

26501365 is choosedn in a way that this turns into $202300 * 131 + 65$ which is especially convenient.

In [9]:
print("Reduce steps to parameters: 26501365 ->",  divmod(26501365, 131))

# Find some of these values in the form n * 131 + 65
found = []
for i in range(1, 10):
    s = bfs(data, start, (131*i) + 65 )
    found.append(len(s))
    print(len(s), end=' ')


Reduce steps to parameters: 26501365 -> (202300, 65)
34075 95592 186147 309182 460107 644660 855955 1102026 1373691 

In [24]:
# Look at succesive differences:
diffs = [b - a for a, b in zip(found, found[1:])]
print(diffs)

# Every other dif is related by multiplication.
[divmod(b, a) for a,b in zip(diffs, diffs[2:])]

[61517, 90555, 123035, 150925, 184553, 211295, 246071, 271665]


[(2, 1), (1, 60370), (1, 61518), (1, 60370), (1, 61518), (1, 60370)]

This is enough to build a rough hypothesis and formula:

In [25]:
def solve(i):
    if i % 2 == 0:
        return 61517 + (i)//2 * 61518
    else:
        return 90555 + (i)//2 * 60370   #!
    

def chain(i):
    total = 34075
    for i in range(i-1):
        total += solve(i)
    return total

print("predict first few n * 131 + 65 values")
print(', '.join(str(chain(i)) for i in range(1, 8)))

print("="*80)
    
result = chain(202300)
print("Part Two:", result-1)

predict first few n * 131 + 65 values
34075, 95592, 186147, 309182, 460107, 644660, 855955
Part Two: 623540829615589
